# DrugEx API

An example DrugEx workflow showcasing some basic DrugEx API features. The API provides interface definitions to handle data operations and training of models needed for obtaining a molecule designer. The interface should ensure that the current code base is extensible and loosely coupled to make interoperability with different data sources seamless and to also aid in monitoring of the training processes involved.

Let's import and explain some of the important API features:

In [9]:
# main package
import drugex

# important classes for data access
from drugex.api.environ.data import ChEMBLCSV
from drugex.api.corpus import CorpusCSV, BasicCorpus, CorpusChEMBL

# important classes for QSAR modelling
# and (de)serialization of QSAR models
from drugex.api.environ.models import RF
from drugex.api.environ.serialization import FileEnvSerializer, FileEnvDeserializer

# classes that handle training of the exploration
# and exploitation networks and also handle monitoring 
# of the process 
from drugex.api.model.callbacks import BasicMonitor
from drugex.api.pretrain.generators import BasicGenerator

# ingredients needed for DrugEx agent training
from drugex.api.agent.agents import DrugExAgent
from drugex.api.agent.callbacks import BasicAgentMonitor
from drugex.api.agent.policy import PG

# designer API (wraps the agent after it was trained)
from drugex.api.designer.designers import BasicDesigner, CSVConsumer

Next let's define some global settings:

In [10]:
import torch

for device in range(torch.cuda.device_count()):
    print(device, torch.cuda.get_device_capability(device))

0 (3, 5)
1 (2, 1)
2 (3, 5)


In [11]:
import os

if torch.cuda.is_available():
    # choose a GPU device based on the info above
    # (the higher the capability, the better)
    torch.cuda.set_device(2)

DATA_DIR="data" # folder with input data files
OUT_DIR="output/workflow" # folder to store the output of this workflow
os.makedirs(OUT_DIR, exist_ok=True) # create the output folder

# define a set of gene IDs that are interesting for 
# the target that we want to design molecules for
GENE_IDS = ["ADORA2A"]

## Data Aquisition

It's time to aquire the data we will need for training of our models. There are three models that we need to build so we need three separate data sets:

1. Data for the exploitation model based on a random sample of 1 million molecules from the ZINC set.
2. Data for the exploration model based on ChEMBL data we downloaded for the desired target.
3. Data for the QSAR modelling of the environment model -> this model will bias the final generator towards more active molecules throug the a policy gradient.

In [ ]:
# Randomly selected sample of 1 million molecules
# from the ZINC database.
# We only use this file for illustration purposes.
# In practice, the pretrained exploitation network will always
# be provided so there will be no need for this data,
# but we are starting from square one here.
ZINC_CSV=os.path.join(DATA_DIR, "ZINC.txt")

# Load structure data into a corpus from a CSV file (we assume
# that we have the structures saved in a csv file in DATA_DIR).
# This corpus will be used to train the exploitation network later.
corpus_pre = CorpusCSV(
    update_file=ZINC_CSV
    # The input CSV file we will use to update the data in this corpus.

    , vocabulary=drugex.VOC_DEFAULT
    # The vocabulary object that defines the tokens
    # and other options used to construct and parse SMILES.
    # VOC_DEFAULT is a reasonable "catch all" default.

    , smiles_column="CANONICAL_SMILES"
    # Tells the corpus object what column to look for when
    # extracting SMILES to update the data.

    , sep='\t'
    # The column separator used in the CSV file
)

# Now we update the corpus (if we did not do it already).
# It loads and tokenizes the SMILES it finds in the CSV.
# The tokenized data and updated vocabulary are returned to us.
corpus_out_zinc = os.path.join(OUT_DIR, "zinc_corpus.txt")
vocab_out_zinc = os.path.join(OUT_DIR, "zinc_voc.txt")
if not os.path.exists(corpus_out_zinc):
    df, voc = corpus_pre.updateData(update_voc=True)
    # We don't really use the return values here, but they are
    # still there if we need them for logging purposes or
    # something else.

    # All that remains is to just save our corpus data
    # so that we don't have to recalculate it again.
    # The CorpusCSV class has a method for that so lets use it.
    corpus_pre.saveCorpus(corpus_out_zinc)
    corpus_pre.saveVoc(vocab_out_zinc)
else:
    # if we already initialized and saved
    # the corpus before, we just overwrite the
    # current one with the saved one
    corpus_pre= CorpusCSV.fromFiles(corpus_out_zinc, vocab_out_zinc)